# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

from datasets import load_dataset

D:\software\anaconda\envs\notebook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
# 0-负面
# 1-正面
print(dataset)

dataset = dataset.filter(lambda x: x["review"] is not None)

print(dataset)

Found cached dataset csv (F:/cache/huggingface/datasets/csv/default-7846a518e13d281d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at F:\cache\huggingface\datasets\csv\default-7846a518e13d281d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ba36305480521887.arrow


Dataset({
    features: ['label', 'review'],
    num_rows: 7766
})
Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})


## Step3 划分数据集

In [4]:
datasets = dataset.train_test_split(test_size=0.1)

datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [5]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    
    tokenized_examples["labels"] = examples["label"]
    
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 创建模型

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

## Step6 创建评估函数

In [10]:
import evaluate

acc_metric = evaluate.load("accuracy")

f1_metirc = evaluate.load("f1")

In [11]:
def eval_metric(eval_predict):
    
    predictions, labels = eval_predict
    
    predictions = predictions.argmax(axis=-1)
    
    acc = acc_metric.compute(predictions=predictions, references=labels)
    
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    
    acc.update(f1)
    
    return acc

## Step7 创建训练参数：TrainingArguments

In [12]:
train_args = TrainingArguments(
    output_dir="./checkpoints",      # 输出文件夹
    per_device_train_batch_size=64,  # 训练时的BATCH_SIZE
    per_device_eval_batch_size=128,  # 验证时的BATCH_SIZE
    logging_steps=10,                # LOG打印的频率
    evaluation_strategy="epoch",     # 评估策略
    save_strategy="epoch",           # 保存策略
    save_total_limit=3,              # 最大保存数
    learning_rate=2e-5,              # 学习率
    weight_decay=0.01,               # WEIGHT_DECAY
    metric_for_best_model="f1",      # 设定评估指标
    load_best_model_at_end=True
)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_toke

## Step8 创建TRAINER

In [13]:
from transformers import DataCollatorWithPadding

trainer = Trainer(
    model=model, 
    args=train_args, 
    train_dataset=tokenized_datasets["train"], 
    eval_dataset=tokenized_datasets["test"], 
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric
)

## Step9 模型训练

In [14]:
trainer.train()

D:\software\anaconda\envs\notebook\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.287800,0.289219,0.877735,0.915103
2,0.226200,0.270912,0.882883,0.916743
3,0.224200,0.270013,0.881596,0.915751


TrainOutput(global_step=330, training_loss=0.2950942429629239, metrics={'train_runtime': 71.1596, 'train_samples_per_second': 294.605, 'train_steps_per_second': 4.637, 'total_flos': 351909933963264.0, 'train_loss': 0.2950942429629239, 'epoch': 3.0})

## Step10 模型评估

In [21]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.27091172337532043,
 'eval_accuracy': 0.8828828828828829,
 'eval_f1': 0.9167429094236048,
 'eval_runtime': 1.0706,
 'eval_samples_per_second': 725.795,
 'eval_steps_per_second': 6.539,
 'epoch': 3.0}

## Step11 模型预测

In [18]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[-2.4537935 ,  1.5196052 ],
       [-1.1916391 ,  0.46352088],
       [ 0.08503188, -0.47836143],
       ...,
       [-2.7789876 ,  1.9177665 ],
       [-3.1554258 ,  2.362263  ],
       [-0.06426902, -0.5774672 ]], dtype=float32), label_ids=array([1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Step6 训练与验证

In [19]:
def evaluate():
    
    model.eval()
    
    with torch.inference_mode():
    
        for batch in validloader:
        
            if torch.cuda.is_available():
            
                batch = {k: v.cuda() for k, v in batch.items()}
            
            output = model(**batch)
            
            pred = torch.argmax(output.logits, dim=-1)
            
            clf_metrics.add_batch(predictions=pred.long(), references=batch["labels"].long())
    
    return clf_metrics.compute()

def train(epoch=3, log_step=100):
    
    global_step = 0
    
    for ep in range(epoch):
    
        model.train()
        
        for batch in trainloader:
        
            if torch.cuda.is_available():
            
                batch = {k: v.cuda() for k, v in batch.items()}
            
            optimizer.zero_grad()
            
            output = model(**batch)
            output.loss.backward()
            
            optimizer.step()
            
            if global_step % log_step == 0:
                
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            
            global_step += 1
        
        clf = evaluate()
        
        print(f"ep: {ep}, {clf}")

## Step7 模型训练

In [20]:
train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\XM\AppData\Local\Temp\ipykernel_19244\3364925475.py:1 in <module>                       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\XM\\AppData\\Local\\Temp\\ipykernel_19244\\3364925475.py'                            │
│                                                                                                  │
│ C:\Users\XM\AppData\Local\Temp\ipykernel_19244\4162449298.py:29 in train                         │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\XM\\AppData\\Local\\Temp\\ipykernel_19244\\4162449298.py'                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainloader' is not defined

## Step8 模型预测

In [11]:
sen = "我觉得这家酒店不错，饭很好吃！"

id2_label = {0: "差评！", 1: "好评！"}

model.eval()

with torch.inference_mode():

    inputs = tokenizer(sen, return_tensors="pt")
    
    inputs = {k: v.cuda() for k, v in inputs.items()}
    
    logits = model(**inputs).logits
    
    pred = torch.argmax(logits, dim=-1)
    
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [12]:
from transformers import pipeline

model.config.id2label = id2_label

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [16]:
pipe(sen)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\XM\AppData\Local\Temp\ipykernel_19244\337104173.py:1 in <module>                        │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\XM\\AppData\\Local\\Temp\\ipykernel_19244\\337104173.py'                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pipe' is not defined